<a href="https://colab.research.google.com/github/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/blob/main/ACND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import pandas as pd
import numpy as np

from google.colab import files
import io
import numpy.ma as ma
from scipy.spatial import distance as dist


In [17]:
data = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/Aggregation.csv", encoding="ISO-8859-1" )

In [18]:
data.head()

,Col_0,Col_1
0,15.55,28.65
1,14.90,27.55
2,14.45,28.35
3,14.15,28.80
4,13.75,28.05


# Etapa 1: Read and analyze data, and determine parameters

In [51]:
N = data.shape[0]
alfa = 0.0
alpha = 0.0
beta = 0.0
gama = 0.0


In [ ]:
for m in data:

  max_s = max(data[m])
  min_s = min(data[m])
  rs = max_s - min_s
  alfa += rs

# α = N/alpha; β = (3.5α + 45)/100;
#γ = (2α + 18)/100; k = max(10, N∗1%);

alpha = N/alfa
beta = ((3.5 * alpha) + 45) / 100
gama = ( (2 * alpha) + 18)/100
k = max(10, (N * 0.01))
print(k)

#Etapa 2: Construct k-D Tree: kDT

In [ ]:
from sklearn.neighbors import KDTree
kdt = KDTree(data)
print(kdt.data.shape)

In [ ]:
kdt.get_arrays()

#Etapa 3: Searches kNN of sample, calculates mean of 5NN’s

In [128]:
distance = []
knn_index = []
d5nn = []

distance, knn_index = kdt.query(data,k)

for i in range(len(knn_index)):
  d5nn.append(np.mean(distance[i, 1:5]))


#Etapa 4: Calculate the global radius of SR

In [129]:
d5nn = np.sort(d5nn)

In [131]:
R = max(d5nn)
R

1.6356889031523352

#Etapa 5: Calculate sample density array(Rho)

In [161]:
def fab(i, j, a, b):
  d = dist.euclidean(i, j)
  if d >= 0 and d <= a:
    return 1
  elif d > a and d <= b:
    return ((b - d)/ (b-a))
  elif d > b:
    return 0

In [ ]:
rho = []
for i in range(N):
  surrounding_region = distance[i,] < R
  surrounding_region = distance[i,surrounding_region]
  surrounding_region = surrounding_region[1:]

  mean_ = np.mean(surrounding_region)
  std_dev = np.std(surrounding_region)
  surrounding_region = np.sort(surrounding_region)
  d1 = 0.1 * surrounding_region
  d2 = min(surrounding_region)
  a = (d1 + d2)/2
  exp = (a + 2*std_dev)
  b = min( min(exp), R)

#   for (j in surrounding-region) {
# f(i, j) = fab(i, j, a, b) // Eq. (5)
# Rho[i] + = f(i, j);}//for j}//for i
  rho.append(0)
  for j in surrounding_region:#range(len(surrounding_region)):
    f = fab(distance[i,1],j,min(a),b)
    rho[i] += f
  print(rho[i])



#Etapa 6: Calculate the global density threshold(T)

In [176]:
# for (i = 0; i < N; i + + ) Arr[i] = (Rho[i], i);
# sort(Arr); // in descending order
# T = lower-Quartile(Arr);
arr = []
for i in range(N):
  arr.append((rho[i], i))

arr = np.sort(arr,0)
arr = arr[::-1]

t = np.percentile(arr, 25) 
t

5.2737409115052625